# Dataset preprocessing

### Synthetic dataset

Before training, `ALIGN_RESULT_v2.zip`, the synthetic dataset provided by the authors, needs to be unzipped to `./datasets/ALIGN_RESULT_v2`.

Since shadow masks are not included in the zip file, use `generate_shadow_label.py` also provided by the authors to generate shadow masks to the same directory:   

In [ ]:
!python D:/CV/final_project/take-off-eyeglasses-main/generate_shadow_label.py --syndata_dir ./datasets/ALIGN_RESULT_v2

### celebA dataset
First unzip `img_celeba` and `list_attr_celeba.txt` and put them under `./datasets/celebA`.
we split images with eyeglasses from the celebA dataset:

In [ ]:
import os
import shutil
import tqdm

real_dir = "./datasets/celebA"

dict_image_glass = {}   # a dictionary that stores whether a image contains eyeglass or not
attribute_list_path = os.path.join(real_dir, "list_attr_celeba.txt")

if not os.path.exists(os.path.join(real_dir, "img_celeba_glass")):
        os.makedirs(os.path.join(real_dir, "img_celeba_glass"))

with open(attribute_list_path, 'r') as f:
            lines = f.readlines()
            eyeglass_attribute_pos = lines[1].split(' ').index("Eyeglasses") + 1
            for line in lines[2:]:
                tmp = line.split(' ')
                tmp = [i for i in tmp if i != '']
                dict_image_glass[tmp[0]] = True if tmp[eyeglass_attribute_pos] == '1' else False
    

for root, _, fnames in sorted(os.walk(os.path.join(real_dir, "img_celeba"))):
            for fname in tqdm.tqdm(fnames):
                if dict_image_glass[fname] == True:
                    src = os.path.join(root, fname)
                    dst = os.path.join(real_dir, "img_celeba_glass")
                    shutil.copy(src, dst)

Then align the images using `align_images.py` from https://github.com/pbaylies/stylegan-encoder :

In [ ]:
!python D:/CV/final_project/stylegan-encoder-master/align_images.py --output_size 256 "./datasets/celebA/img_celeba_glass" "./datasets\celebA/img_align_celeba_glass_256x256"

# Mask Prediction Stage


### Training
Use `--continue_train` to continue from previous progress

In [ ]:
!python DA_train.py --experiment 'DA_train_1' --batch_size 8 --epochs 30 --random_seed 123 --log_iter 20 --display_iter 80 --ckpt_iter 240 --synetic_root './datasets/ALIGN_RESULT_v2' --celebA_root './datasets/celebA/img_align_celeba_glass_256x256' --pth_root './checkpoints'

# Item Removal Stage

### Prepare updated synthetic dataset
Generate `ALIGN_RESULT_v2_our_mask` dataset for this stage, which ontains original, shadow-removed, and glass-removed images from synetic dataset and glass/shadow masks generated from previously trained mask prediction stage

In [ ]:
!python mask_inference.py --synetic_root './datasets/ALIGN_RESULT_v2' --pth_root './checkpoints/DA_train_1' --save_root './inference_output/ALIGN_RESULT_v2_our_mask'

### Training
Use `--continue_train` to continue from previous progress

In [ ]:
!python removal_train.py --experiment 'removal_train_1' --batch_size 8 --epochs 80 --random_seed 123 --log_iter 20 --display_iter 180 --ckpt_iter 207 --synetic_root './inference_output/ALIGN_RESULT_v2_our_mask' --pth_root './checkpoints'